# 1. Import of input data

In [ ]:
import nbimporter
import sys
import os
from pathlib import Path

In [ ]:
# Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CombinedHeatPower_WasteToEnergy.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 CombinedHeatPower_WasteToEnergy.ipynb executed standalone — using directory: {current_dir}")

# Percorso robusto di Libraries.ipynb
Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# Se non trovato, prova nei livelli superiori
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Esegui il file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

In [ ]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'CHP-Waste-to-Energy')

# 2. Input parameters

In [ ]:
ElectricEnergyproduced = energysystem['Electric energy produced in a year (exported and consumed)[kWh /year]']
ThermalEnergyproduced_primary = energysystem['Thermal energy produced in a year [kWh /year]']
Waste = energysystem['Energy input from burning waste residue [kWh/year]']
NaturalGas = energysystem['Energy input from natural gas [kWh/year]']
SupplyTemperature = energysystem['Supply temperature [°C]']
ReturnTemperature = energysystem['Return temperature [°C]']
Power = energysystem['Thermal capacity [kW]']
EfficiencyW = energysystem['Electrical efficiency based on LHV (between 0 and 1)']
EfficiencyQ = energysystem['Heat efficiency based on LHV (between 0 and 1)']
EquivalentHours = ThermalEnergyproduced_primary / Power

In [ ]:
LHVbiowaste = 2.50 # kWh / kg
LHVnonbiowaste = 3.50 #kWh / kg
x = np.array(((Waste / LHVbiowaste + LHVnonbiowaste)) / 100000) #100000 They are the tons of waste burned by the Swiss system taken as a reference #np.ones(len(Power))
Lifespan = 40 # years

In [ ]:
# ADDITIONAL CALCULATIONS
EfficiencyTot = EfficiencyW + EfficiencyQ
EnergyInput = (Power * EquivalentHours * RSP) / EfficiencyQ
TotalW = EnergyInput * EfficiencyW
TotalQ = EnergyInput * EfficiencyQ

In [ ]:
AFQ_WtE = 0
AFW_WtE = 0

# 3. Life cycle inventories

## 3.1 Module A1-3

In [ ]:
# MODULE A1-3
WtE_cement = (4990000 * x) * AFQ_WtE
WtE_gravel = (35000000 * x) * AFQ_WtE
WtE_pitch = (800000 * x) * AFQ_WtE
WtE_reinforcingsteel = (3920000 * x) * AFQ_WtE
WtE_sand = (10000000 * x) * AFQ_WtE
WtE_chromiumsteel = (80000 * x) * AFQ_WtE

mass = (
    WtE_cement +
    WtE_gravel +
    WtE_pitch +
    WtE_reinforcingsteel +
    WtE_sand +
    WtE_chromiumsteel
)

## 3.2 Module A4

In [ ]:
mass = np.array(mass)
WtE_Truk_16_32 = ((500 + 200)*(mass)/1000) * AFQ_WtE
WtE_Light_Truk = (50*(mass)/1000) * AFQ_WtE

## 3.3 Module B2

In [ ]:
Lifespan = np.array(Lifespan)
WtE_LightTruk_B2 = (50 * (Lifespan/2) * 0.01 * (mass/1000)) * AFQ_WtE

## 3.4 Module B4

In [ ]:
WtENumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFWtE = (RSP / ReqSL)  # Scaling factor

In [ ]:
Efficiency_LHV = EfficiencyW

# DEFAULT PARAMETERS
LHV = 36.8
HHV = 40.4

Efficiency_HHV = Efficiency_LHV*(LHV/HHV)  # Efficiency based on HHV

NaturalGas = np.array([NaturalGas])  # Energy need for 1 year kWh per year

NaturalGas = NaturalGas.flatten()

y = (NaturalGas * 3.6 * RSP).flatten() * (1 + Losses)

# REPARTITION IN THE ENERGY SOURCE
# Costants
k = 0.0256410256410256

# Base terms calculations
base = (1 / Efficiency_HHV) / (k * HHV)
h2_term = (1 / (Efficiency_HHV * (12.7 / 0.09)))

conditions = [
    (Year == 2020),
    (Year == 2025),
    (Year == 2030),
    (Year == 2035),
    (Year == 2040),
    (Year == 2045),
    (Year == 2050),
]

values = [
    base,
    base * 0.95 + base * 0.042 + h2_term * 0.007,
    base * 0.90 + base * 0.084 + h2_term * 0.014,
    base * 0.795 + base * 0.189 + h2_term * 0.016,
    base * 0.688 + base * 0.294 + h2_term * 0.018,
    base * 0.582 + base * 0.399 + h2_term * 0.020,
    base * 0 + base * 0.25 + h2_term * 0.20,
]

Gas = np.select(conditions, values, default=np.nan)

## 3.5 Module B6

In [ ]:
WtE_EnergyVectorNG = (((1 / (Efficiency_HHV * HHV))) * Gas * y) * A * AFQ_WtE
WtE_EnergyVectorBiomethane = ((1 / (Efficiency_HHV * HHV)) * Gas * y) * B * AFQ_WtE
WtE_EnergyVectorHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * Gas * y) * C * AFQ_WtE #120 MJ/kg

WtE_Waste = np.array(((Waste / LHVbiowaste + LHVnonbiowaste)) * RSP) * AFQ_WtE

WtE_municipalsolidwaste = (0 * Waste * 1000 * 50) * AFQ_WtE

## 3.6 Module C2

In [ ]:
WtE_Transport_to_treatment_plant = 50 * (mass/1000) * AFQ_WtE

## 3.7 Module C3

In [ ]:
WtE_EOL_cement = WtE_cement
WtE_EOL_gravel = WtE_gravel
WtE_EOL_pitch = WtE_pitch
WtE_EOL_reinforcingsteel = WtE_reinforcingsteel
WtE_EOL_sand = WtE_sand
WtE_EOL_chromiumsteel = WtE_chromiumsteel

## 3.8 Module C4

In [ ]:
WtE_EOL_landfill_cement = WtE_EOL_cement * LandfillInert
WtE_EOL_landfill_gravel = WtE_EOL_gravel * LandfillInert
WtE_EOL_landfill_pitch = WtE_EOL_pitch * LandfillInert
WtE_EOL_landfill_reinforcingsteel = WtE_EOL_reinforcingsteel * LandfillMetals
WtE_EOL_landfill_sand = WtE_EOL_sand * LandfillInert
WtE_EOL_landfill_chromiumsteel = WtE_EOL_chromiumsteel * LandfillMetals2

## 3.9 Module D1

In [ ]:
WtE_EOL_recycling_reinforcingsteel = (RecyclingMetals * RecyclingEfficiencyMetals * WtE_EOL_reinforcingsteel) - (RecycledContentMetals * WtE_EOL_reinforcingsteel)
WtE_EOL_primary_reinforcingsteel = ((RecycledContentMetals * WtE_EOL_reinforcingsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * WtE_EOL_reinforcingsteel))* SRmetals

WtE_EOL_recycling_chromiumsteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * WtE_EOL_chromiumsteel) - (RecycledContentMetals * WtE_EOL_chromiumsteel)
WtE_EOL_primary_chromiumsteel = ((RecycledContentMetals * WtE_EOL_chromiumsteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * WtE_EOL_chromiumsteel))* SRmetals

# 4. Data for export

In [ ]:
ThermalEnergyProduced_WtE = np.array([TotalQ])